# Formula 1: Co2 Calculations

This notebook will calculate the amount of fuel used by F1 cars now and in the past to work out how much Co2 has been saved. Then we will calculate potential Co2 savings from optimizing the season race schedule to save as much Co2 as possible.

In [26]:
# Import Libraties
import pandas as pd
import matplotlib as plt
import requests

 Input Data
 
 All sources provided, change sources and variables (below) to update all the calculations

- F1 Fuel website 
https://motorsportexplained.com/f1-fuel/
-  Co2 per litre of fuel https://www.carbonindependent.org/17.html#:~:text=The%20CO%202%20emissions%20from%20petrol%20and%20diesel,per%20gallon%20%28since%201%20gallon%20is%204.546%20litres%29.

In [25]:
# Variables
fuel_qty_2019 = 110
cars_per_race = 22
fuel_co2_litre = 2.4



In [13]:
# Max fuel used per race
total_race_fuel = fuel_qty_2019*cars_per_race
print(total_race_fuel)

2420


In [24]:
# Max Co2 per race (from fuel use)
co2_race2019 = total_race_fuel*fuel_co2_litre
print("During the 2019 season a single F1 race emits", co2_race2019, "kg of Co2.")

During the 2019 season a single F1 race emits 5808.0 kg of Co2.


In [47]:
page_url_season_2023 = 'https://en.wikipedia.org/wiki/2023_Formula_One_World_Championship'

html = requests.get(page_url_season_2023, timeout = 5, verify = True)

html.text[:1000]

dfs2023 = pd.read_html(html.text)

dfs2023[1]


,Round,Grand Prix,Circuit,Race date
0,1,Bahrain Grand Prix,"Bahrain International Circuit, Sakhir",5 March
1,2,Saudi Arabian Grand Prix,"Jeddah Corniche Circuit, Jeddah",19 March
2,3,Australian Grand Prix,"Albert Park Circuit, Melbourne",2 April
3,4,Azerbaijan Grand Prix,"Baku City Circuit, Baku",30 April
4,5,Miami Grand Prix,"Miami International Autodrome, Miami Gardens, ...",7 May
5,6,Monaco Grand Prix,"Circuit de Monaco, Monaco",28 May
6,7,Spanish Grand Prix,"Circuit de Barcelona-Catalunya, Montmeló",4 June
7,8,Canadian Grand Prix,"Circuit Gilles Villeneuve, Montreal",18 June
8,9,Austrian Grand Prix,"Red Bull Ring, Spielberg",2 July
9,10,British Grand Prix,"Silverstone Circuit, Silverstone",9 July


In [50]:
df = dfs2023[1]
df.head()

,Round,Grand Prix,Circuit,Race date
0,1,Bahrain Grand Prix,"Bahrain International Circuit, Sakhir",5 March
1,2,Saudi Arabian Grand Prix,"Jeddah Corniche Circuit, Jeddah",19 March
2,3,Australian Grand Prix,"Albert Park Circuit, Melbourne",2 April
3,4,Azerbaijan Grand Prix,"Baku City Circuit, Baku",30 April
4,5,Miami Grand Prix,"Miami International Autodrome, Miami Gardens, ...",7 May


In [51]:
df.to_csv('season2023.csv', index = True)